### ATR 3X Strategy

In [14]:
import time
from datetime import datetime
import vectorbt as vbt

import pandas as pd
import pandas_ta as ta
import numpy as np
import mplfinance as mpf

### Import CSV

In [15]:
data = pd.read_csv('GBPUSD_15m.csv')
data.tail()

,timestamp,open,high,low,close,adj close,volumn
11920,2022-04-29 16:45:00+01:00,1.256329,1.256929,1.256187,1.256581,1.256581,0
11921,2022-04-29 17:00:00+01:00,1.256329,1.256644,1.255824,1.255950,1.255950,0
11922,2022-04-29 17:15:00+01:00,1.256281,1.257719,1.256281,1.256787,1.256787,0
11923,2022-04-29 17:30:00+01:00,1.256266,1.257150,1.256266,1.256913,1.256913,0
11924,2022-04-29 17:45:00+01:00,1.256992,1.257656,1.256676,1.257450,1.257450,0


In [16]:
df = data.copy()

### Create strategy

In [4]:
atr_3x = ta.Strategy(
    name = 'Gloden cross',
    description = 'Gloden cross by EMA 2 Line',
    ta = [
        {'kind': 'atr', 'length': 14},
        {'kind': 'ema', 'length': 260}
    ]
)
df.ta.strategy(atr_3x)
df

,timestamp,open,high,low,close,adj close,volumn,ATRr_14,EMA_260
0,2021-11-04 17:00:00+00:00,113.750000,113.750999,113.690002,113.730003,113.730003,0,NaN,NaN
1,2021-11-04 17:05:00+00:00,113.732002,113.739998,113.712997,113.731003,113.731003,0,NaN,NaN
2,2021-11-04 17:10:00+00:00,113.731003,113.745003,113.690002,113.724998,113.724998,0,NaN,NaN
3,2021-11-04 17:15:00+00:00,113.726997,113.731003,113.660004,113.692001,113.692001,0,NaN,NaN
4,2021-11-04 17:20:00+00:00,113.688004,113.693001,113.629997,113.670998,113.670998,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
35636,2022-04-29 17:35:00+01:00,129.779999,129.834000,129.759995,129.809998,129.809998,0,0.117115,130.211475
35637,2022-04-29 17:40:00+01:00,129.807007,129.811996,129.729996,129.766998,129.766998,0,0.114607,130.208069
35638,2022-04-29 17:45:00+01:00,129.772995,129.783997,129.710007,129.737000,129.737000,0,0.111705,130.204460
35639,2022-04-29 17:50:00+01:00,129.735001,129.768005,129.639999,129.669998,129.669998,0,0.112870,130.200364


### Create signal

In [17]:
class BackTestATR3X:
    def __init__(self, params):
        self.orders = pd.DataFrame(columns = ['action', 'open time', 'open', 'close time', 'close', 'T/P', 'S/L', 'result'])
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        self.unknow_result = []
        
        self.current_action = 'close'
        self.atr = params['atr']
        self.ema = params['ema']
        
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        self.df = params['df']
        self.plot = params['plot']

    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
    def get_df(self):
        return self.df
    
    def plot_order(self, order):
        pass
    
    def backtest(self):
        atr_3x = ta.Strategy(
            name = 'Gloden cross',
            description = 'Gloden cross by EMA 2 Line',
            ta = [
                {'kind': 'atr', 'length': self.atr},
                {'kind': 'ema', 'length': self.ema}
            ]
        )
        self.df.ta.strategy(atr_3x)
        self.df = self.df.copy().dropna()
        self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
        self.df['action'] = None
        
        self.df['entries'] = False
        self.df['exits'] = False
        self.df['short_entries'] = False
        self.df['short_exits'] = False
        self.df['price_action'] = self.df['close']
        
        for index, row in self.df.iterrows():
            entry = 0
            
            if self.current_action == 'close':
                if (row['EMA_'+str(self.ema)] < row['close']) and (row['ATRr_'+str(self.atr)]*3 < (row['high']-row['low']) ):
                    self.df.at[index, 'action'] = 'buy'
                    self.df.at[index, 'entries'] = True
                    self.df.at[index, 'price_action'] = row['open']
                    self.order['action'] = 'buy'
                                
                    self.order['open time'] = row['timestamp']
                    self.order['open'] = row['open']
                    self.order['S/L'] = row['open'] - row['ATRr_'+str(self.atr)]
                    self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                    self.current_action = 'buy'
                    
                elif (row['EMA_'+str(self.ema)] > row['close']) and (row['ATRr_'+str(self.atr)]*3 < (row['high']-row['low']) ):
                    self.df.at[index, 'action'] = 'sell'
                    self.df.at[index, 'short_entries'] = True
                    self.df.at[index, 'price_action'] = row['open']
                    self.order['action'] = 'sell'
                                
                    self.order['open time'] = row['timestamp']
                    self.order['open'] = row['open']
                    self.order['S/L'] = row['open'] + row['ATRr_'+str(self.atr)]
                    self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                    self.current_action = 'sell'
        
            if self.current_action == 'buy':
                if row['low'] <= self.order['S/L'] and row['high'] >= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['low'] <= self.order['S/L']:
                    self.df.at[index, 'exits'] = True
                    self.df.at[index, 'price_action'] = row['low']
                    
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()
                    
                elif row['high'] >= self.order['T/P']:
                    self.df.at[index, 'exits'] = True
                    self.df.at[index, 'price_action'] = row['high']
                    
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()

            elif self.current_action == 'sell':
                if row['high'] >= self.order['S/L'] and row['low'] <= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['high'] >= self.order['S/L']:
                    self.df.at[index, 'short_exits'] = True
                    self.df.at[index, 'price_action'] = row['high']
                    
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()
                    
                elif row['low'] <= self.order['T/P']:
                    self.df.at[index, 'short_exits'] = True
                    self.df.at[index, 'price_action'] = row['low']
                    
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()

        try:
            win_rate = round(len(self.orders[self.orders['result'] == 'T/P']) * 100 / len(self.orders), 2)
            loss_rate = round(len(self.orders[self.orders['result'] == 'S/L']) * 100 / len(self.orders), 2)

            win_orders = int((win_rate * len(self.orders)) / 100 )
            loss_orders = len(self.orders) - win_orders
            gain = (win_orders * self.rr) - loss_orders

            result_info = f"total orders: {len(self.orders)}\nema: {self.ema}, atr: {self.atr}, gain: {gain}\n"
            return {'win_rate': win_rate, 'info': result_info, 'total_order': len(self.orders), 'rr': self.rr, 'gain': gain, 'unknow': self.unknow_result}
        except:
    #         print("No orders to action")
            return {'win_rate': 0, 'info': 'No orders to action'}

In [18]:
pair_test = BackTestATR3X({
    'atr': 14,
    'ema': 260,
    'pip_value': 0.001,
    'rr': 1.5,
    'plot': False,
    'df': df.copy()
})
pair_test.backtest()

{'win_rate': 57.89,
 'info': 'total orders: 19\nema: 260, atr: 14, gain: 6.0\n',
 'total_order': 19,
 'rr': 1.5,
 'gain': 6.0,
 'unknow': [timestamp        2021-12-15 19:00:00+00:00
  open                              1.320167
  high                              1.321545
  low                               1.317731
  close                             1.321161
  adj close                         1.321161
  volumn                                   0
  ATRr_14                            0.00104
  EMA_260                           1.323264
  action                                None
  entries                              False
  exits                                False
  short_entries                        False
  short_exits                          False
  price_action                      1.321161
  Name: 2747, dtype: object,
  timestamp        2022-04-06 19:00:00+01:00
  open                              1.307668
  high                              1.309843
  low                   

In [19]:
"""
Grid parameter example
ema: 200 -> 260
atr: 10 -> 30
condition line 1 < line 2 < line 3
"""
def create_grid_params(line1_start, line1_end, line2_start, line2_end):
    params = []
    for i in range(line1_start, line1_end+1):
        for j in range(line2_start, line2_end+1):
            params.append([i, j])
    return params
params_list = create_grid_params(200, 260, 10, 30)
len(params_list)

1281

In [20]:
good_params = {'win': 0, 'info': ''}
start = 0
end = 1281
win_start = 0

loop_num = start
for param in params_list[start:end]:
    obj = BackTestATR3X({'atr': param[1], 'ema': param[0], 'pip_value': 0.001, 'rr': 1.5, 'df': df.copy(), 'plot': False})
    result = obj.backtest()
    if result['win_rate'] > win_start and result['total_order'] > 100:
        win_start = result['win_rate']
        good_params['win'] = result['win_rate']
        good_params['info'] = result['info']
    loop_num = loop_num + 1
    print(f"run: {loop_num} to {end}", end = "\r")
    
print(f"\nwin rate: {good_params['win']} %\n{good_params['info']}")

run: 1281 to 1281
win rate: 0 %

